In [ ]:
import pandas as pd
import time
import requests

IMAGE_DOWNLOAD_DIRECTORY = '/Users/baskausj/Downloads/act_images/'

# Read in clean metadata CSV as text into dataframe, with NA values as empty strings
metadata_path = '../processed_lists/clean_metadata_2022-09-29.csv'
act_metadata_df = pd.read_csv(metadata_path, dtype=str, na_filter=False)

# Find all rows whose lower case "CopyrightStatement" column value equals "image donated by Jim Womack and Anne Richardson" or variant
richardson_df1 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'image donated by Jim Womack and Anne Richardson']
richardson_df2 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'Image donated by Jim Womack and Anne Richardson']
richardson_df3 = act_metadata_df.loc[act_metadata_df['CopyrightStatement'] == 'Image donated by Anne Richardson']
# Combine the two dataframes
richardson_df = pd.concat([richardson_df1, richardson_df2, richardson_df3])

# Set the RecordNumber column as the index
richardson_df.set_index('RecordNumber', inplace=True)

# Save the slice as a CSV
#richardson_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'richardson.csv')

# Slice the values from the ImageLink column and create a new dataframe with them, using the same index as the original dataframe
filenames_df = richardson_df.loc[:, 'ImageLink']
filenames_df = pd.DataFrame(filenames_df)

# Add a column named file_url
filenames_df['file_url'] = ''

# Save the slice as a CSV
filenames_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv')

filenames_df.head()


In [ ]:
# In case a script crash requires restarting, reload the CSV file
filenames_df = pd.read_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv', dtype=str, na_filter=False)

# Set the index to the RecordNumber column
filenames_df = filenames_df.set_index('RecordNumber')

# Loop through the rows and download the images from the URL
for index, row in filenames_df.iterrows():
    # If the file_url column is not empty, skip this row
    if row['file_url'] != '':
        continue

    # Convert the filename to a string
    filename_string = row['ImageLink']

    # If the filename contains only numeric characters, padd it on the left with 0s until it has 8 characters
    if filename_string.isnumeric():
        filename_string = filename_string.zfill(8)
    url = 'https://diglib.library.vanderbilt.edu/cdri/fulljpeg/' + filename_string + '.jpg'
    print(url)
    
    # Add the file URL to the dataframe
    filenames_df.at[index, 'file_url'] = url

    # Download the image
    filename = filename_string + '.jpg'
    r = requests.get(url)
    with open(IMAGE_DOWNLOAD_DIRECTORY + filename, 'wb') as outfile:
        outfile.write(r.content)

    # Wait 0.5 second before downloading the next image
    time.sleep(0.5)

    # Write the dataframe to a CSV file after each download
    # to keep track of which ones were done in case the script crashes.
    filenames_df.to_csv(IMAGE_DOWNLOAD_DIRECTORY + 'filenames.csv')
    